In [1]:
using PowerSystems
using Dates
using TimeSeries
using DataFrames
using CSV
using DataStructures
using PowerSystemCaseBuilder
using PowerSimulations
using HiGHS
using HydroPowerSimulations

In [2]:
"""
"name":"HydroPumpedStorage","ramp_limits_pump":{"up":1.0,"down":1.0},"status":"OFF","active_power":0.0,"storage_target":{"up":0.0,"down":0.0},"dynamic_injector":null,"__metadata__":{"module":"PowerSystems","type":"HydroPumpedStorage"},"active_power_limits_pump":{"min":0.0,"max":1.0},"internal":{"uuid":{"value":"ccd0458b-77ee-4f3d-b065-37562dc08168"},"ext":null,"units_info":null},"reactive_power_limits":{"min":0.0,"max":1.0},"rating_pump":1.0,"ext":{},"operation_cost":{"fixed":0.0,"variable":{"__metadata__":{"parameters":["InputOutputCurve"],"module":"InfrastructureSystems","type":"CostCurve"},"value_curve":{"input_at_zero":null,"function_data":{"__metadata__":{"module":"InfrastructureSystems","type":"LinearFunctionData"},"constant_term":0.0,"proportional_term":0.15},"__metadata__":{"parameters":["LinearFunctionData"],"module":"InfrastructureSystems","type":"InputOutputCurve"}},"power_units":"NATURAL_UNITS","vom_cost":{"input_at_zero":null,"function_data":{"__metadata__":{"module":"InfrastructureSystems","type":"LinearFunctionData"},"constant_term":0.0,"proportional_term":0.0},"__metadata__":{"parameters":["LinearFunctionData"],"module":"InfrastructureSystems","type":"InputOutputCurve"}}},"__metadata__":{"module":"PowerSystems","type":"HydroGenerationCost"}},"reactive_power_limits_pump":null,"time_limits":null,"conversion_factor":1.0,"pump_efficiency":0.8,"storage_capacity":{"up":2.0,"down":2.0},"ramp_limits":{"up":0.1,"down":0.1},"time_at_status":10000.0,"active_power_limits":{"min":0.0,"max":1.0},"reactive_power":0.0,"inflow":0.0,"outflow":0.0},{"rating":11.148,"services":[],"x":0.0108,"b":{"from":0.00926,"to":0.00926},"r":0.00108,"available":true,
"""

"\"name\":\"HydroPumpedStorage\",\"ramp_limits_pump\":{\"up\":1.0,\"down\":1.0},\"status\":\"OFF\",\"active_power\":0.0,\"storage_target\":{\"up\":0.0,\"down\":0.0},\"dynamic_injector\":null,\"__metadata__\":{\"module\":\"PowerSystems\",\"type\":\"HydroPumpedStorage\"},\"active_power_limits_pump\":{\"min\":0" ⋯ 1138 bytes ⋯ "own\":2.0},\"ramp_limits\":{\"up\":0.1,\"down\":0.1},\"time_at_status\":10000.0,\"active_power_limits\":{\"min\":0.0,\"max\":1.0},\"reactive_power\":0.0,\"inflow\":0.0,\"outflow\":0.0},{\"rating\":11.148,\"services\":[],\"x\":0.0108,\"b\":{\"from\":0.00926,\"to\":0.00926},\"r\":0.00108,\"available\":true,\n"

In [3]:
# from https://github.com/NREL-Sienna/PowerSystems.jl/blob/main/docs/src/tutorials/creating_system.md
sys = System(100.0)

bus1 = ACBus(;
    number = 1,
    name = "bus1",
    bustype = ACBusTypes.REF,
    angle = 0.0,
    magnitude = 1.0,
    voltage_limits = (min = 0.9, max = 1.05),
    base_voltage = 230.0,
);

bus4 = ACBus(;
    number = 4,
    name = "bus4",
    bustype = ACBusTypes.PV,
    angle = 0.0,
    magnitude = 1.0,
    voltage_limits = (min = 0.9, max = 1.05),
    base_voltage = 230.0,
);

line41 = Line(;
    name = "line41",
    available = true,
    active_power_flow = 0.0,
    reactive_power_flow = 0.0,
    arc = Arc(; from = bus4, to = bus1),
    r = 0.00281, # Resistance, Per-unit
    x = 0.0281, # Reactance, Per-unit
    b = (from = 0.00356, to = 0.00356), # Shunt susceptance, Per-unit
    rating = 4.0, # Line rating of 200 MVA / System base of 100 MVA
    angle_limits = (min = -0.7, max = 0.7),
);

load =  PowerLoad(;
    name = "load",
    available = true,
    bus = bus1,
    active_power = 0.0, # Per-unitized by device base_power
    reactive_power = 0.0, # Per-unitized by device base_power
    base_power = 100.0, # MVA
    max_active_power = 1.0, # 10 MW per-unitized by device base_power
    max_reactive_power = 0.0,
);

# https://github.com/NREL-Sienna/PowerSystems.jl/blob/ec2c55991de76d25deb0c9ac23d85e44979d9694/src/models/generated/HydroEnergyReservoir.jl
hydro = HydroPumpedStorage(;
    name = "hydro",
    available = true,
    bus = bus4,
    active_power = 0.0,
    reactive_power = 0.0,
    active_power_limits = (min=0.0, max=1.0), # MW from 2 28,000 HP pumps https://www.sdcwa.org/wp-content/uploads/2020/11/lake_hodges_fs.pdf?q=/sites/default/files/files/publications/lakehodges-fs.pdf&
    rating = 1.0, # "Maximum power withdrawal (MVA) of the pump"
    prime_mover_type = PrimeMovers.PS,
    reactive_power_limits = (min=0.0, max=1.0),
    rating_pump = 11.148, # MVA
    active_power_limits_pump = (min=0.0, max=1.0),
    reactive_power_limits_pump = (min=0.0, max=1.0),
    ramp_limits_pump = (up=1.0, down=1.0), # "ramp up and ramp down limits in MW/min of pump"
    time_limits_pump = nothing, # "Minimum up and Minimum down time limits of pump in hours"
    inflow = 0, # "Baseline inflow into the upper reservoir (units can be p.u. or m^3/hr)"
    outflow = 0, # "Baseline outflow from the lower reservoir (units can be p.u. or m^3/hr)"
    initial_storage = (up=1, down=0), # "Initial storage capacity in the upper and lower reservoir (units can be p.u-hr or m^3)"
    ramp_limits=(up=1, down=1), # "ramp up and ramp down limits in MW/min"
    time_limits=(up=0.0, down=0), # "Minimum up and Minimum down time limits in hours"
    operation_cost = HydroGenerationCost(CostCurve(LinearCurve(0)), 0),
    base_power = 100.0, # MVA
    storage_capacity = (up=2, down=2)
    )

add_components!(sys, [bus1, bus4, line41, load, hydro])

In [4]:
fname = joinpath(homedir(), "ecen5407", "ecen5407_project2", "data", "Sienna_Inputs", "sienna_pointers.json");
add_time_series!(sys, fname)

┌ Warning: no component category=Generator name=solar
└ @ InfrastructureSystems /home/emco4286/.julia/packages/InfrastructureSystems/rXaFr/src/system_data.jl:697
┌ Warning: no component category=Generator name=wind
└ @ InfrastructureSystems /home/emco4286/.julia/packages/InfrastructureSystems/rXaFr/src/system_data.jl:697


5-element Vector{TimeSeriesKey}:
 StaticTimeSeriesKey(SingleTimeSeries, "max_active_power", DateTime("2024-11-18T00:00:00"), Millisecond(300000), 105120, Dict{String, Any}())
 StaticTimeSeriesKey(SingleTimeSeries, "outflow", DateTime("2024-11-18T00:00:00"), Millisecond(300000), 105120, Dict{String, Any}())
 StaticTimeSeriesKey(SingleTimeSeries, "inflow", DateTime("2024-11-18T00:00:00"), Millisecond(300000), 105120, Dict{String, Any}())
 StaticTimeSeriesKey(SingleTimeSeries, "storage_capacity", DateTime("2024-11-18T00:00:00"), Millisecond(300000), 105120, Dict{String, Any}())
 StaticTimeSeriesKey(SingleTimeSeries, "max_active_power", DateTime("2024-11-18T00:00:00"), Millisecond(300000), 105120, Dict{String, Any}())

In [5]:
transform_single_time_series!(
    sys,
    Dates.Hour(24), # horizon
    Dates.Hour(1), # interval
);

In [6]:
template_ed = ProblemTemplate(CopperPlatePowerModel)
set_device_model!(template_ed, HydroPumpedStorage, HydroDispatchPumpedStorage)
solver = optimizer_with_attributes(HiGHS.Optimizer, "mip_rel_gap" => 0.5)

model = DecisionModel(
    template_ed,
    sys,
    name="ED",
    optimizer=solver,
    optimizer_solve_log_print=true,
    store_variable_names=true,
)

Network Model
┌───────────────┬───────────────────────┐
│ Network Model │ CopperPlatePowerModel │
│ Slacks        │ false                 │
│ PTDF          │ false                 │
│ Duals         │ None                  │
└───────────────┴───────────────────────┘

Device Models
┌────────────────────┬────────────────────────────┬────────┐
│ Device Type        │ Formulation                │ Slacks │
├────────────────────┼────────────────────────────┼────────┤
│ HydroPumpedStorage │ HydroDispatchPumpedStorage │ false  │
└────────────────────┴────────────────────────────┴────────┘

In [7]:
build!(model, output_dir = mktempdir())

InfrastructureSystems.Optimization.ModelBuildStatusModule.ModelBuildStatus.BUILT = 0

In [8]:
solve!(model)

Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [8e-02, 1e+00]
  Cost   [0e+00, 0e+00]
  Bound  [1e+00, 2e+00]
  RHS    [1e+00, 1e+00]
Assessing feasibility of MIP using primal feasibility and integrality tolerance of       1e-06
Solution has               num          max          sum
Col     infeasibilities      0            0            0
Integer infeasibilities      0            0            0
Row     infeasibilities      0            0            0
Row     residuals            0            0            0
Presolving model
576 rows, 863 cols, 1724 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               0     

InfrastructureSystems.Simulation.RunStatusModule.RunStatus.SUCCESSFULLY_FINALIZED = 0

In [9]:
res = OptimizationProblemResults(model)

HydroEnergyOutput__HydroPumpedStorage
ProductionCostExpression__HydroPumpedStorage
ActivePowerBalance__System
OutflowTimeSeriesParameter__HydroPumpedStorage
InflowTimeSeriesParameter__HydroPumpedStorage
ActivePowerInVariable__HydroPumpedStorage
HydroEnergyVariableUp__HydroPumpedStorage
HydroEnergyVariableDown__HydroPumpedStorage
ReservationVariable__HydroPumpedStorage
ActivePowerOutVariable__HydroPumpedStorage
WaterSpillageVariable__HydroPumpedStorage


In [10]:
list_parameter_names(res)

2-element Vector{String}:
 "OutflowTimeSeriesParameter__HydroPumpedStorage"
 "InflowTimeSeriesParameter__HydroPumpedStorage"

In [13]:
read_parameter(res, "InflowTimeSeriesParameter__HydroPumpedStorage")

Row,DateTime,hydro
,DateTime,Float64
1,2024-11-18T00:00:00,0.0
2,2024-11-18T00:05:00,0.0
3,2024-11-18T00:10:00,0.0
4,2024-11-18T00:15:00,0.0
5,2024-11-18T00:20:00,0.0
6,2024-11-18T00:25:00,0.0
7,2024-11-18T00:30:00,0.0
8,2024-11-18T00:35:00,0.0
9,2024-11-18T00:40:00,0.0


In [10]:
var = read_variables(res)
var.keys

16-element Memory{String}:
    "HydroEnergyVariableUp__HydroPumpedStorage"
 #undef
 #undef
    "ReservationVariable__HydroPumpedStorage"
 #undef
    "WaterSpillageVariable__HydroPumpedStorage"
 #undef
    "HydroEnergyVariableDown__HydroPumpedStorage"
 #undef
    "ActivePowerInVariable__HydroPumpedStorage"
 #undef
 #undef
 #undef
 #undef
    "ActivePowerOutVariable__HydroPumpedStorage"
 #undef

In [11]:
describe(var["ActivePowerOutVariable__HydroPumpedStorage"])

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Any,Any,Int64,DataType
1,DateTime,,2024-11-18T00:00:00,2024-11-18T11:57:30,2024-11-18T23:55:00,0,DateTime
2,hydro,0.0,0.0,0.0,0.0,0,Float64


In [12]:
describe(var["ActivePowerOutVariable__HydroPumpedStorage"])

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Any,Any,Int64,DataType
1,DateTime,,2024-11-18T00:00:00,2024-11-18T11:57:30,2024-11-18T23:55:00,0,DateTime
2,hydro,0.0,0.0,0.0,0.0,0,Float64
